In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from langdetect import DetectorFactory
DetectorFactory.seed = 0
from utils.marking_similars import *
from utils.translation import parallelize_dataframe,add_features
parent_path='../Data/New_Data_15-06-2020/'

In [ ]:
import glob
annotation_data_receive='../Data/Annotations_received/*'
files=glob.glob(annotation_data_receive)

In [4]:
import parmap
thresh_to_consider = 0.7
signature_obj=Signature(10)


In [5]:
def get_list(temp,row):
    signature1=row['signatures']
    temp_list=[]
    for index1, row1 in temp.iterrows():
#         if row1['orig_index'] in already_seen:
#             continue
        signature2=row1['signatures']
        count = 0
        # Count the number of positions in the minhash signature which are equal.
        if(abs(len(row['message_text'])-len(row1['message_text']))>1000):
            continue
        for k in range(0, signature_obj.numHashes):
            count = count + (signature1[k] == signature2[k])
        # add to tuple similar if greater than thresh    
        if((count/signature_obj.numHashes)>thresh_to_consider):
            temp_list.append(row1['orig_index'])
            #print("found")
    return temp_list

def remove_duplicates_within(sample_df):
    sample_df=add_signatures(sample_df,signature_obj.numHashes)
    id_done=0
    duplicate={}
    for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
        id_done+=1
        try:
            temp_dupl=duplicate[index]
            continue
        except KeyError:
            temp=sample_df[id_done:]
            for index1,row1 in temp.iterrows():
                try:
                    temp_dupl=duplicate[index]
                    continue
                except KeyError:
                    signature1=row['signatures']
                    signature2=row1['signatures']
                    count=0
                    for k in range(0, signature_obj.numHashes):
                        count = count + (signature1[k] == signature2[k])
                    # add to tuple similar if greater than thresh    
                    if((count/signature_obj.numHashes)>thresh_to_consider):
                          duplicate[index1]=1

    
    sample_df=sample_df.drop(list(duplicate.keys()))
    return sample_df


def remove_duplicates_ext(sample_df,total_annotation):
    df_total=pd.concat([sample_df,total_annotation])
    df_total=add_signatures(df_total,signature_obj.numHashes)
    sample_df=df_total[0:len(sample_df)]
    total_annotation=df_total[len(sample_df):]
    duplicate={}
    for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
        for index1,row1 in total_annotation.iterrows():
            signature1=row['signatures']
            signature2=row1['signatures']
            count=0
            for k in range(0, numHashes):
                count = count + (signature1[k] == signature2[k])
            # add to tuple similar if greater than thresh    
            if((count/signature_obj.numHashes)>thresh_to_consider):
                duplicate[index]=1
    sample_df=sample_df.drop(list(duplicate.keys()))
    return sample_df
 


def get_similar_from_whole_data(sample_df,total_annotation,n_process=30):
    df_total=pd.concat([sample_df,total_annotation])
    df_total=add_signatures(df_total,10)
    sample_df=df_total[0:len(sample_df)]
    total_annotation=df_total[len(sample_df):]
    all_repeated=[]
    count_missing=0
    for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
        df_split = np.array_split(total_annotation, n_process)
        list_all = parmap.map(get_list, df_split,row,pm_processes=n_process)
        flat_list = [item for sublist in list_all for item in sublist]
        if(len(flat_list)==0):
            count_missing+=1
            print(count_missing)
        all_repeated.append(flat_list)
        
    sample_df['repeated']=all_repeated
    return sample_df,count_missing
 

In [6]:
files

['../Data/Annotations_received/annotation_recv_6.pkl',
 '../Data/Annotations_received/annotation_recv_1.pkl',
 '../Data/Annotations_received/annotation_recv_8.pkl',
 '../Data/Annotations_received/annotation_recv_3.pkl',
 '../Data/Annotations_received/annotation_recv_2.pkl',
 '../Data/Annotations_received/annotation_recv_9.pkl',
 '../Data/Annotations_received/annotation_recv_5.pkl',
 '../Data/Annotations_received/annotation_recv_7.pkl',
 '../Data/Annotations_received/annotation_recv_0.pkl',
 '../Data/Annotations_received/annotation_recv_10.pkl',
 '../Data/Annotations_received/annotation_recv_4.pkl']

### Steps to be done
1. Read the current data 
2. match signature set for the new data
3. get the new data to find duplicates

In [7]:
list_df=[]
for file in files:
    df=pd.read_pickle(file)
    annotated_df=df[~((df['Annotation1']=='No_Annotation') | (df['Annotation2']=='No_Annotation') | (df['Annotation3']=='No_Annotation'))]
    list_df.append(annotated_df)
total_df = pd.concat(list_df)

In [31]:
total_df=remove_duplicates_within(total_df)
print(len(total_df))

100%|██████████| 5353/5353 [00:09<00:00, 564.32it/s]

2562491



5336


In [ ]:
total_df=remove_duplicates_ext(total_df,annotated_df)

100%|██████████| 9892/9892 [00:17<00:00, 559.40it/s]

4857007


In [44]:
len(total_df)

780

In [46]:
whatsapp_data=pd.read_csv(parent_path+'Data_text_spam_removed_v02.csv')
temp=whatsapp_data[['group_id_anonymized','phone_num_anonymized','message_text','timestamp']]
duplicateDFRow = temp[temp.duplicated()]
whatsapp_data=whatsapp_data.drop(list(duplicateDFRow.index))


In [ ]:
total_df=get_similar_from_whole_data(total_df,whatsapp_data,n_process=30)

In [63]:
total_df=pd.concat([annotated_df,total_df[0]])

In [64]:
def return_atleast_one_count(df):
    dict_map={'Fear speech':1,'Normal':0, 'No_Annotation':2}
    fear_count=[]
    for index,row in df.iterrows():
        annotation_list=[0,0,0]
        annotation_list[dict_map[row['Annotation1']]]+=1
        annotation_list[dict_map[row['Annotation2']]]+=1
        annotation_list[dict_map[row['Annotation3']]]+=1
        if(annotation_list[1]>=2):
            fear_count.append(1)
        elif(annotation_list[0]>=2):
            fear_count.append(0)
        else:
            fear_count.append(-1)
        #listM.append(annotation_list)
    df['one_fear_speech']=fear_count
    #sample_df=df[df['one_fear_speech']==1]
    return df

In [65]:
sample_df=return_atleast_one_count(total_df)

In [5]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data.pkl')

In [22]:
from langdetect import detect

def add_language(df):
    language= []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            lang=detect(row["message_text"])
            language.append(lang)
        except KeyError:
            language.append('unk')

    df["language"] = language
    return df

In [23]:
annotated_df=add_language(annotated_df)

100%|██████████| 5338/5338 [01:10<00:00, 75.33it/s] 


In [27]:
from tqdm import tqdm_notebook,tqdm
size=100
list_df=[]
for i in tqdm(range(0,len(annotated_df),size)):
    print(i,"_iteration")
    df_new=parallelize_dataframe(annotated_df[i:i+size],add_features,n_cores=40)
    list_df.append(df_new)
    df_translated=pd.concat(list_df,ignore_index=True)

  0%|          | 0/54 [00:00<?, ?it/s]

0 _iteration
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
<class 'selenium.common.exceptions.WebDriverException'> Message: unknown error: Cannot set property 'value' of null
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.36.540471 (9c759b81a907e70363c6312294d30b6ccccc2752),platform=Linux 4.15.0-111-generic x86_64)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


  2%|▏         | 1/54 [00:38<34:17, 38.83s/it]

100 _iteration
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
<class 'selenium.common.exceptions.WebDriverException'> Message: unknown error: Cannot set property 'value' of null
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.36.540471 (9c759b81a907e70363c6312294d30b6ccccc2752),platform=Linux 4.15.0-111-generic x86_64)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


  4%|▎         | 2/54 [01:21<34:34, 39.90s/it]

...

 96%|█████████▋| 52/54 [40:38<01:38, 49.28s/it]

5200 _iteration
<class 'selenium.common.exceptions.SessionNotCreatedException'> Message: session not created exception
from disconnected: unable to connect to renderer
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.36.540471 (9c759b81a907e70363c6312294d30b6ccccc2752),platform=Linux 4.15.0-111-generic x86_64)

<class 'selenium.common.exceptions.SessionNotCreatedException'> Message: session not created exception
from disconnected: unable to connect to renderer
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.36.540471 (9c759b81a907e70363c6312294d30b6ccccc2752),platform=Linux 4.15.0-111-generic x86_64)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


 98%|█████████▊| 53/54 [41:28<00:49, 49.56s/it]

5300 _iteration
done
done
<class 'selenium.common.exceptions.WebDriverException'> Message: unknown error: Cannot set property 'value' of null
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.36.540471 (9c759b81a907e70363c6312294d30b6ccccc2752),platform=Linux 4.15.0-111-generic x86_64)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


100%|██████████| 54/54 [41:47<00:00, 40.40s/it]


0       hi
1       hi
2       hi
3       hi
4       hi
        ..
5333    hi
5334    hi
5335    hi
5336    hi
5337    hi
Name: language, Length: 5338, dtype: object

In [37]:
df_translated.to_pickle(parent_path+'Fearspeech_data.pkl')

In [ ]:
sample_df['label']=sample_df['one_fear_speech']

In [ ]:
sample_df[sample_df['label'].isin([0,1])][['message_text','label']].to_csv(path+'Fearspeech_Data.csv')

In [ ]:
sample_fear=sample_df[sample_df['label'].isin([0,1])][['message_text','label']]

In [ ]:
sample_fear.to_pickle(path+'Fearspeech_Data.pickle')

In [80]:
import numpy as np

countrepeat_list=[]
count_missing=0
for index,row in fear_speech_df.iterrows():
    if(len(row['repeated'])>0):
       countrepeat_list.append(len(row['repeated']))
    else:
       count_missing+=1

,Annotation1,Annotation2,Annotation3,User1,User2,User3,forwarded,group_id_anonymized,group_type,hindu_keyword_count,...,pred_probab,preds,repeated,timestamp,tokenized,translated,unique_ids,signatures,one_fear_speech,present_in_data
9701,Fear speech,Fear speech,Normal,Adarsh,Yuvraj,Vineeth,1.0,7.0,BJP,1.0,...,0.030909,0.0,"[26904.0, 5559349.0]",1.550186e+12,"[प्रशासक, समिति, raised_fist, triangular_flag,...",NaN,"[bf7a79ae-f605-4b16-86a8-0fab7d1d8dc8, 8dff263...","[24811490, 31121478, 43379428, 3407718, 214666...",1,1
21649,Normal,Normal,Normal,Adarsh,Yuvraj,Vineeth,1.0,7.0,BJP,1.0,...,0.004815,0.0,"[58653.0, 2538100.0, 2543772.0, 2841047.0, 284...",1.550921e+12,"[हैदराबाद, की, मक्का, मस्जिद, में, ब्लास्ट, मर...",NaN,"[7ee2ce95-7967-4514-ad12-e082cd80d994, 0511697...","[9153108, 62225323, 88426676, 135771602, 30037...",0,1
25630,Fear speech,Fear speech,Normal,Divyanshu,Adarsh,Vineeth,1.0,51.0,BJP,0.0,...,0.684925,1.0,"[69294.0, 520737.0, 521637.0, 534993.0, 608583...",1.551149e+12,"[(, ), ``, इस्लाम, धर्म, नहीं, एक, मानसिक, रोग...",NaN,"[90bc3ab7-0148-4761-9756-d54592a270b2, 9b89d01...","[11378459, 12793291, 38427369, 27223211, 10805...",1,1
33159,Fear speech,Normal,Normal,Divyanshu,Sharat,Kukkadapu,1.0,30.0,BJP,0.0,...,0.055631,0.0,[89540.0],1.551532e+12,"[पाकिस्तान, से, अधिक, तो, जमातएइस्लामी, के, सं...",NaN,"[e230abbc-1869-4614-852c-e9d2f34b2a0a, 14946c4...","[478024745, 699792695, 51911929, 329347590, 35...",0,1
44073,Normal,Normal,Fear speech,Sharat,Adarsh,Vineeth,1.0,75.0,BJP,13.0,...,0.014175,0.0,"[118131.0, 118521.0, 118540.0, 118548.0, 11865...",1.552123e+12,"[शहरी, नक्सलवाद, माओवाद, ।, बस, एक, निवेदन, पढ...",NaN,"[96f6b93f-a84e-43fb-bfdc-7082c2f75a59, e3b7d48...","[8414089, 5483401, 2054993, 5239771, 4023459, ...",0,1


In [76]:
list_present=[]
for index,row in annotated_df.iterrows():
    if(len(row['repeated'])>0):
        list_present.append(1)
    else:
        list_present.append(0)

In [77]:
annotated_df['present_in_data']=list_present

In [78]:
annotated_df_present=annotated_df[annotated_df['present_in_data']==1]

In [79]:
from laserembeddings import Laser
from os import path
import pandas as pd
from tqdm import tqdm_notebook
import demoji
demoji.download_codes()


... OK (Got response in 2.18 seconds)
Writing emoji data to /home/punyajoy/.demoji/codes.json ...
... OK


In [ ]:
list_text = []
lang_text = []
for index,row in tqdm_notebook(annotated_df_present.iterrows(),total=len(annotated_df_present)):
    if len(row['message_text'])>4000:
        string=row['message_text'][0:2000]+row['message_text'][-2000:]
        list_text.append(demoji.replace(string,repl=""))
    else:
        list_text.append(demoji.replace(row["message_text"],repl=""))
    lang_text.append(row["language"])

In [ ]:
laser = Laser()

embed_list=[]

length_given=len(list_text)
batch_size=64
for i in tqdm_notebook(range(0,length_given,batch_size)):
    if(i+batch_size<=length_given):
        temp_text=list_text[i:i+batch_size]
        temp_lang=lang_text[i:i+batch_size]
    else:
        temp_text=list_text[i:length_given]
        temp_lang=lang_text[i:length_given]
    embeddings = laser.embed_sentences(temp_text,lang=temp_lang)  # lang is only used for tokenization
    embed_list+=list(embeddings)

In [ ]:
annotated_df_present.columns

In [ ]:
X_0 = np.array(embed_list)
y_0 = np.array(annotated_df_present['one_fear_speech'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import random

In [ ]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]

skf = StratifiedKFold(n_splits=10, random_state= 2020)

for train_index, test_index in skf.split(X_0, y_0):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_0[train_index], X_0[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    print(class_weights)
    classifier= LogisticRegression(class_weight=class_weights)
    classifier.fit(X_train, y_train)
    y_pred=classifier.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test))


In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))


In [ ]:
num_spreader=[]
for index,row in tqdm_notebook(annotated_df_present.iterrows(),total=len(annotated_df_present)):
    pd.set_option('display.max_colwidth', 300)
    temp=whatsapp_data[whatsapp_data['orig_index'].isin(row['repeated'])]
    if(row['one_fear_speech']==1):
        num_spreader.append([index,len(list(temp['phone_num_anonymized'].unique())),'Fear speech'])
    else:
        num_spreader.append([index,len(list(temp['phone_num_anonymized'].unique())),'Normal'])


In [ ]:
df_num_users=pd.DataFrame(num_spreader,columns=['msg_id','users who spread','label'])
ax1 = sns.barplot(x="label", y="users who spread", data=df_num_users)

In [ ]:
message_length_avg=[]
for index,row in tqdm_notebook(annotated_df_present.iterrows(),total=len(annotated_df_present)):
    if(row['one_fear_speech']==1):
        try:
            message_length_avg.append([index,len(row['tokenized']),'Fear speech'])
        except:
            message_length_avg.append([index,0,'Fear speech'])

    else:
        try:
            
            message_length_avg.append([index,len(row['tokenized']),'Normal'])
        except:
            message_length_avg.append([index,0,'Normal'])



In [ ]:
df_num_users=pd.DataFrame(message_length_avg,columns=['msg_id','avg_message_length(in words)','label'])
ax1 = sns.barplot(x="label", y='avg_message_length(in words)', data=df_num_users)

In [ ]:
from urlextract import URLExtract
import tldextract
ext = tldextract.extract('www.twitter.co/MF6mvzSBDs?ssr=true')
print(ext.subdomain, ext.domain, ext.suffix)


extractor = URLExtract()
urls = extractor.find_urls("Text with URLs. Let's have URL janlipovsky.cz as an example.")
print(urls)

In [ ]:
def return_url_dict(df):
    count=0
    url_dict={}
    for text in tqdm_notebook(df.message_text):
        urls = extractor.find_urls(text)
        if(len(urls)>0):
            for url in urls:
                splited=url.split('/')
                if(len(splited)>=3 and splited[2]=='youtu.be'):
                    urlkey='youtube.com'
                elif(len(splited)>=3 and splited[2]=='t.co'):
                    urlkey='telegram.com'
                elif(len(splited)>=3 and splited[2]=='g.co'):
                    urlkey='google.com'
                elif(len(splited)>=3 and splited[2]=='wp.me'):
                    urlkey='wordpress.com'
                elif(len(splited)>=3 and splited[2]=='bit.ly'):
                    continue
                else:
                    ext = tldextract.extract(url)
                    urlkey=ext.domain+'.'+ext.suffix
                try:
                    url_dict[urlkey]+=1
                except:
                    url_dict[urlkey]=1
            count+=1
    print(count)
    return url_dict

In [ ]:
fear_speech_url_dict=return_url_dict(annotated_df_present[annotated_df_present['one_fear_speech']==1])

In [ ]:
normal_url_dict=return_url_dict(annotated_df_present[annotated_df_present['one_fear_speech']==0])

In [ ]:
def rank_order_difference(dict1,dict2,max_rank=40):
    tuples_A=sorted(dict1.items(), key=lambda item: item[1],reverse=True)
    tuples_B=sorted(dict2.items(), key=lambda item: item[1],reverse=True)
    
    dict_A={}
    count=0
    for ele in tuples_A:
        if(count<max_rank):
            count+=1
        dict_A[ele[0]]=count
    
    dict_B={}
    count=0
    for ele in tuples_B:
        if(count<max_rank):
            count+=1
        dict_B[ele[0]]=count
    
    rank_diff_A={}
    for key in dict_A.keys():
        try:
            rank_diff_A[key]=dict_B[key]-dict_A[key]
        except:
            rank_diff_A[key]=max_rank-dict_A[key]
    rank_diff_B={}
    for key in dict_B.keys():
        try:
            rank_diff_B[key]=dict_A[key]-dict_B[key]
        except:
            rank_diff_B[key]=max_rank-dict_B[key]
            
    return rank_diff_A,rank_diff_B 

In [ ]:
rank_diff_A,rank_diff_B =rank_order_difference(fear_speech_url_dict,normal_url_dict,20)

In [ ]:
list_hashtag=sorted(rank_diff_A.items(), key=lambda item: item[1],reverse=True)[0:10]
[ele[0] for ele in list_hashtag]

In [ ]:
list_hashtag=sorted(rank_diff_B.items(), key=lambda item: item[1],reverse=True)[0:10]
[ele[0] for ele in list_hashtag]

In [ ]:
import emoji

import re
import emoji
def extract_emojis(str1):
    try:
        return [c for c in str1 if c in emoji.UNICODE_EMOJI]
    except AttributeError:
        return []

def extract_emojis_index(str1):
    try:
        return [(i,c) for i,c in enumerate(str1) if c in emoji.UNICODE_EMOJI]
    except AttributeError:
        return []

In [ ]:
emoji_dict_normal={}
count_emoji_present=0
for index,row in tqdm_notebook(annotated_df_present[annotated_df_present['one_fear_speech']==0].iterrows(),total=annotated_df_present[annotated_df_present['one_fear_speech']==0].shape[0]):
    #print(row["message_text"])
    list_emojis=set(extract_emojis(row['message_text']))
    if(len(list_emojis)>0):
        count_emoji_present+=1
    #print(list_emojis)
    
    for emoji_this in list_emojis:
        try:
            emoji_dict_normal[emoji_this]+=1
        except KeyError:
            emoji_dict_normal[emoji_this]=1


In [ ]:
count_emoji_present

In [ ]:
emoji_dict_fear={}
count_emoji_present=0
for index,row in tqdm_notebook(annotated_df_present[annotated_df_present['one_fear_speech']==1].iterrows(),total=annotated_df_present[annotated_df_present['one_fear_speech']==1].shape[0]):
    #print(row["message_text"])
    list_emojis=set(extract_emojis(row['message_text']))
    #print(list_emojis)
    if(len(list_emojis)>0):
        count_emoji_present+=1
    
    for emoji_this in list_emojis:
        try:
            emoji_dict_fear[emoji_this]+=1
        except KeyError:
            emoji_dict_fear[emoji_this]=1


In [ ]:
count_emoji_present

In [ ]:
rank_diff_A,rank_diff_B =rank_order_difference(emoji_dict_fear,emoji_dict_normal,20)

In [ ]:
list_hashtag=sorted(rank_diff_B.items(), key=lambda item: item[1],reverse=True)[0:10]
[ele[0] for ele in list_hashtag]

In [ ]:
list_hashtag=sorted(rank_diff_A.items(), key=lambda item: item[1],reverse=True)[0:10]
[ele[0] for ele in list_hashtag]